In [1]:
import pandas as pd
import numpy as np

staging_df = pd.read_csv("database/API_19_DS2_en_csv_v2_4644.csv")

print("\nDados brutos (staging):")
print(staging_df.head())
print("\nColunas do DataFrame:", staging_df.columns.tolist())



Dados brutos (staging):
  Country Name Country Code                            Indicator Name  \
0        Aruba          ABW  Urban population (% of total population)   
1        Aruba          ABW                          Urban population   
2        Aruba          ABW        Urban population growth (annual %)   
3        Aruba          ABW                         Population, total   
4        Aruba          ABW              Population growth (annual %)   

      Indicator Code       1960          1961          1962          1963  \
0  SP.URB.TOTL.IN.ZS     50.776     50.761000     50.746000     50.730000   
1        SP.URB.TOTL  27887.000  28212.000000  28580.000000  28917.000000   
2        SP.URB.GROW        NaN      1.158679      1.295975      1.172248   
3        SP.POP.TOTL  54922.000  55578.000000  56320.000000  57002.000000   
4        SP.POP.GROW        NaN      1.187344      1.326227      1.203664   

           1964          1965  ...           2016           2017  \
0    

Transformação e limpeza dos dados

In [2]:
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
year_cols = [str(year) for year in range(2000, 2025)]
value_vars = [col for col in year_cols if col in staging_df.columns]

cleaned_df = pd.melt(
    staging_df,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='current_year',
    value_name='current_value'
)

cleaned_df.dropna(subset=['current_value'], inplace=True)
cleaned_df = cleaned_df[cleaned_df['Country Name'] != 'World']
cleaned_df['current_year'] = pd.to_numeric(cleaned_df['current_year']).astype(int)
cleaned_df['current_value'] = pd.to_numeric(cleaned_df['current_value']).astype(float)

cleaned_df.rename(columns={
        'Country Name': 'country_name',
        'Country Code': 'country_code',
        'Indicator Name': 'indicator_name',
        'Indicator Code': 'indicator_code'
    }, inplace=True)

print("\nDados limpos e transformados:")
print(cleaned_df.head())


Dados limpos e transformados:
  country_name country_code                            indicator_name  \
0        Aruba          ABW  Urban population (% of total population)   
1        Aruba          ABW                          Urban population   
2        Aruba          ABW        Urban population growth (annual %)   
3        Aruba          ABW                         Population, total   
4        Aruba          ABW              Population growth (annual %)   

      indicator_code  current_year  current_value  
0  SP.URB.TOTL.IN.ZS          2000      46.717000  
1        SP.URB.TOTL          2000   42320.000000  
2        SP.URB.GROW          2000       0.156076  
3        SP.POP.TOTL          2000   90588.000000  
4        SP.POP.GROW          2000       1.030817  


Datawarehouse

In [3]:
dim_country = cleaned_df[['country_name', 'country_code']].drop_duplicates().sort_values('country_name').reset_index(drop=True)
dim_country['id_country'] = dim_country.index
dim_country = dim_country[['id_country', 'country_name', 'country_code']]

print("\ntabela da dim_country")
print(dim_country.head())


tabela da dim_country
   id_country                 country_name country_code
0           0                  Afghanistan          AFG
1           1  Africa Eastern and Southern          AFE
2           2   Africa Western and Central          AFW
3           3                      Albania          ALB
4           4                      Algeria          DZA


In [4]:
dim_indicator = cleaned_df[['indicator_name', 'indicator_code']].drop_duplicates().sort_values('indicator_name').reset_index(drop=True)
dim_indicator['id_indicator'] = dim_indicator.index
dim_indicator = dim_indicator[['id_indicator', 'indicator_name', 'indicator_code']]

print("\ntabela da dim_indicator")
print(dim_indicator.head())


tabela da dim_indicator
   id_indicator                                     indicator_name  \
0             0            Access to electricity (% of population)   
1             1  Agricultural irrigated land (% of total agricu...   
2             2                 Agricultural land (% of land area)   
3             3                         Agricultural land (sq. km)   
4             4  Agriculture, forestry, and fishing, value adde...   

      indicator_code  
0     EG.ELC.ACCS.ZS  
1  AG.LND.IRIG.AG.ZS  
2     AG.LND.AGRI.ZS  
3     AG.LND.AGRI.K2  
4     NV.AGR.TOTL.ZS  


In [5]:
dim_time = cleaned_df[['current_year']].drop_duplicates().sort_values('current_year').reset_index(drop=True)
dim_time['id_time'] = dim_time.index
dim_time = dim_time[['id_time', 'current_year']]

print("\n--- Tabela de Dimensão: dim_time ---")
print(dim_time.head())


--- Tabela de Dimensão: dim_time ---
   id_time  current_year
0        0          2000
1        1          2001
2        2          2002
3        3          2003
4        4          2004


In [6]:
fat_value_temp = cleaned_df.merge(dim_country, on='country_code')
fat_value_temp = fat_value_temp.merge(dim_indicator, on='indicator_code')
fat_value_temp = fat_value_temp.merge(dim_time, on='current_year')

fat_value = fat_value_temp[['id_country', 'id_indicator', 'id_time', 'current_value']].copy()
    
fat_value.drop_duplicates(subset=['id_country', 'id_indicator', 'id_time'], inplace=True)
    
fat_value.reset_index(drop=True, inplace=True)
fat_value['id_value'] = fat_value.index
    
fat_value = fat_value[['id_value', 'id_country', 'id_indicator', 'id_time', 'current_value']]

print("\ntabela de fat_value")
print(fat_value.head())

print(f"\nProcesso ETL concluído. Foram gerados {len(fat_value)} registros na tabela de fatos.")


tabela de fat_value
   id_value  id_country  id_indicator  id_time  current_value
0         0          12            48        0      46.717000
1         1          12            47        0   42320.000000
2         2          12            49        0       0.156076
3         3          12            33        0   90588.000000
4         4          12            30        0       1.030817

Processo ETL concluído. Foram gerados 198017 registros na tabela de fatos.


In [7]:
fat_value.to_csv("datawarehouse/fat_value.csv", index=False)
dim_country.to_csv("datawarehouse/dim_country.csv", index=False)
dim_indicator.to_csv("datawarehouse/dim_indicator.csv", index=False)
dim_time.to_csv("datawarehouse/dim_time.csv", index=False)